# Strategy

> Selecting the system's behavior at run-time

In this scenario we'll implement a simple text processor that will output format in either HTML or Markdown. For simplicity's sake, the only complex element we'll process is a list.

We'll begin with the text processor itself. It will have a buffer that will contain all the text. Let's also add a method that will process a collection of items into a list in the appropiate format.

In [1]:
class TextProcessor:
    def __init__(self):
        self.buffer = []
        
    def append_list(self, items):
        pass # we'll fully implement it later

We need to tell the `TextProcessor` the format we want the list to be in. We want to make our solution flexible, so we will define a **strategy** that will be used to process a list.

We will define a `ListStrategy` base class as an interface for our strategies. A base class is not required but it's always good practice.

In [2]:
from abc import ABC

class ListStrategy(ABC):
    def start(self, buffer):
        """Appends the start tag of the list to the buffer"""
        pass

    def end(self, buffer):
        """Appends the end tag of the list to the buffer"""
        pass

    def add_list_item(self, buffer, item):
        """Add an item to the buffer with their corresponding tags"""
        pass

Let's implement our strategies:
* Markdown does not require either start or end tags for lists but it does prefix each item within a list with an asterisk `*`.
* HTML needs tags at the start and end of both the list and the elements (start and end tag for each element).

In [3]:
class MarkdownListStrategy(ListStrategy):
    def add_list_item(self, buffer, item):
        buffer.append(f' * {item}\n')

class HtmlListStrategy(ListStrategy):
    def start(self, buffer):
        buffer.append('<ul>\n')

    def end(self, buffer):
        buffer.append('</ul>\n')

    def add_list_item(self, buffer, item):
        buffer.append(f'  <li>{item}</li>\n') # note the indentation

> Note that we could be violating the Interface Segregation Principle by having too many methods in the interface (`MarkdownListStrategy` only uses 1 of the 3 methods), but we're being flexible because in this particular instance this code is useful.

We can now go back to the `TextProcessor` class and expand it. We'll use `HtmlListStrategy` by default and we'll store the strategy in the instance.

For `append_list`, we'll simply loop through the list and add the tags on each item, as well as making sure that we add the appropiate start/end tags for the list before and after the loop.

We'll also create a new `self_output_format` method that will define the output format; we'll be able to change the format dynamically thanks to the **Strategy Design Pattern**. We will need to make use of an enum for the strategies inside this method.

Finally, we'll add some utility methods for clearing up the buffer as well as string representation for the `TextProcessor` class.

In [ ]:
from enum import Enum, auto

class OutputFormat(Enum):
    MARKDOWN = auto()
    HTML = auto()

class TextProcessor:
    def __init__(self, list_strategy=HtmlListStrategy()):
        self.buffer = []
        self.list_strategy = list_strategy
        
    def append_list(self, items):
        self.list_strategy.start(self.buffer)
        for item in items:
            self.list_strategy.add_list_item(
                self.buffer, item
            )
        self.list_strategy.end(self.buffer)
        
    def set_output_format(self, format):
        """Here is where the Strategy Design Pattern magic happens"""
        if format == OutputFormat.MARKDOWN:
            self.list_strategy = MarkdownListStrategy()
        elif format == OutputFormat.HTML:
            self.list_strategy = HtmlListStrategy()
            
    def clear(self):
        self.buffer.clear()

    def __str__(self):
        return ''.join(self.buffer)

We're now ready to test our code. We'll output a list in a format and then change the format at runtime and output the list again.

In [5]:
items = ['foo', 'bar', 'baz']

tp = TextProcessor()
tp.set_output_format(OutputFormat.MARKDOWN)
tp.append_list(items)
print(tp)

print('---')

# let's change the output format at runtime
tp.set_output_format(OutputFormat.HTML)
tp.clear()
tp.append_list(items)
print(tp)

 * foo
 * bar
 * baz

---
<ul>
  <li>foo</li>
  <li>bar</li>
  <li>baz</li>
</ul>



What we've seen in this scenario is that we can have a separate piece of code that works a a blueprint for an algorithm (`ListStrategy`) which we can then implement in different ways according to our needs. This is useful in scenarios such as unit testing, where you can test different parts using dummy strategies.

> Note that the Strategy design pattern has some similaries to the Factory design pattern: both involve interfaces and classes being selected dynamically. But Factory pattern is meant for creating generic objects and delegating the implementation choice to the factory, whereas the Strategy pattern is meant for changing the behavior at runtime by having a selection of interchangeable algorithms that can be passed to a context object without changing it.